In [1]:
import pandas as pd
import numpy as np
# Get some time series data
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/timeseries.csv")
df.head()

,Date,A,B,C,D,E,F,G
0,2008-03-18,24.68,164.93,114.73,26.27,19.21,28.87,63.44
1,2008-03-19,24.18,164.89,114.75,26.22,19.07,27.76,59.98
2,2008-03-20,23.99,164.63,115.04,25.78,19.01,27.04,59.61
3,2008-03-25,24.14,163.92,114.85,27.41,19.61,27.84,59.41
4,2008-03-26,24.44,163.45,114.84,26.86,19.53,28.02,60.09


In [3]:
# Put your inputs into a single list
df['single_input_vector'] = df[['A','B']].apply(tuple, axis=1).apply(list)
# Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
df['single_input_vector'] = df.single_input_vector.apply(lambda x: [list(x)])
# Use .cumsum() to include previous row vectors in the current row list of vectors
df['cumulative_input_vectors'] = df.single_input_vector.cumsum()

In [9]:
df['output_vector'] = df[['G']].apply(tuple, axis=1).apply(list)


In [11]:
# Pad your sequences so they are the same length
from keras.preprocessing.sequence import pad_sequences

max_sequence_length = df.cumulative_input_vectors.apply(len).max()
# Save it as a list   
padded_sequences = pad_sequences(df.cumulative_input_vectors.tolist(), max_sequence_length).tolist()
df['padded_input_vectors'] = pd.Series(padded_sequences).apply(np.asarray)

In [14]:
# Extract your training data
X_train_init = np.asarray(df.padded_input_vectors)
# Use hstack to and reshape to make the inputs a 3d vector
X_train = np.hstack(X_train_init).reshape(len(df),max_sequence_length,2)
y_train = np.hstack(np.asarray(df.output_vector)).reshape(len(df),1)

In [18]:
print(X_train_init.shape)
print(X_train.shape)
print(X_train == X_train_init)

(11,)
(11, 11, 2)
False


/home/andris/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
input_length = X_train.shape[1]
input_dim = X_train.shape[2]
output_dim = len(y_train[0])


In [20]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense

# Build the model
model = Sequential()

# I arbitrarily picked the output dimensions as 4
model.add(LSTM(4, input_dim = input_dim, input_length = input_length))
# The max output value is > 1 so relu is used as final activation.
model.add(Dense(output_dim, activation='relu'))

model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])

/home/andris/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/home/andris/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(4, input_shape=(11, 2))`
  


In [24]:
X_train

array([[[  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [ 24, 164]],

       [[  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [ 24, 164],
        [ 24, 164]],

       [[  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [ 24, 164],
        [ 24, 164],
        [ 23, 164]],

       [[  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [ 24, 164],
        [ 24, 164],
        [ 23, 164],
        [ 24, 163]],

       [[  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0,   0],
        [  0

In [21]:
history = model.fit(X_train, y_train,
              batch_size=7, nb_epoch=3,
              verbose = 1)


/home/andris/anaconda/lib/python3.6/site-packages/keras/models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/3
11/11 [==============================] - 0s 28ms/step - loss: 3376.2530 - acc: 0.0000e+00
Epoch 2/3
11/11 [==============================] - 0s 2ms/step - loss: 2968.9096 - acc: 0.0000e+00
Epoch 3/3
11/11 [==============================] - 0s 2ms/step - loss: 2611.4049 - acc: 0.0000e+00
